In [1]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import time
from util import fast_mapk

In [2]:
#get data
df_train = pd.read_csv( "clicks_train.csv") #100,000
events = pd.read_csv( "events.csv") #100,000

In [3]:
#split train test by display id
t=time.time()
msk = np.random.rand(len(df_train.display_id.unique())) < 0.8
msk=df_train.display_id.unique()[msk]
print time.time()-t
t=time.time()
train = df_train[df_train.display_id.isin([msk])]
print time.time()-t
t=time.time()
val = df_train[~df_train.display_id.isin([msk])]
print time.time()-t

2.9632499218
10.0849268436
8.67872405052


In [18]:
import pickle
pkl_file = open('/datadrive/le.pickle', 'rb')
le=pickle.load(pkl_file)

In [19]:
le

{'ad_id': LabelEncoder(),
 'document_id': LabelEncoder(),
 'geo_location': LabelEncoder(),
 'uuid': LabelEncoder()}

In [22]:
import time
t=time.time()
joined_train_clicks=train.merge(events)
print time.time()-t
t=time.time()
joined_val_clicks=val.merge(events)
print time.time()-t

16.483412981
14.9806599617


In [25]:
len(events.uuid.unique())

19794967

In [26]:
len(le['uuid'].classes_)

19794962

In [46]:
joined_train_clicks=joined_train_clicks[~joined_train_clicks['uuid']\
                                        .isin(['5cda9845a1b9be', '7e8aa06b36db6a' ,'83e9ec48908c6a', 'f4bb634c3871b9'])]
joined_val_clicks=joined_val_clicks[~joined_val_clicks['uuid']\
                                        .isin(['5cda9845a1b9be', '7e8aa06b36db6a' ,'83e9ec48908c6a', 'f4bb634c3871b9','558ba104e8a37c'])]

In [47]:
#set features
ad_ids=joined_train_clicks['ad_id'].append(joined_val_clicks['ad_id'])
uuids=joined_train_clicks['uuid'].append(joined_val_clicks['uuid'])

In [44]:
#convert feautres to int
column_diffs=[]
t=time.time()
train_users=le['uuid'].transform(joined_train_clicks['uuid'])
test_users=le['uuid'].transform(joined_val_clicks['uuid'])

NameError: name 'joined_test_clicks' is not defined

In [49]:
column_diffs=[]
t=time.time()
train_users=le['uuid'].transform(joined_train_clicks['uuid'])
test_users=le['uuid'].transform(joined_val_clicks['uuid'])
column_diffs.append(max(np.append(train_users,test_users)))
print time.time()-t
t=time.time()

test_ads=le['ad_id'].transform(joined_val_clicks['ad_id'])+column_diffs[0]
train_ads=le['ad_id'].transform(joined_train_clicks['ad_id'])+column_diffs[0]
column_diffs.append(max(np.append(test_ads,train_ads)))
print time.time()-t
t=time.time()

5.11455488205
27.5681819916


In [67]:
#generate final data frames
t=time.time()
final_frame=pd.DataFrame({'uuid':train_users,'ad_id':train_ads,'clicked':joined_train_clicks['clicked'].astype(int)})
print time.time()-t
t=time.time()
final_val_frame=pd.DataFrame({'uuid':test_users,'ad_id':test_ads,'clicked':joined_val_clicks['clicked'].astype(int)})
print time.time()-t

0.652989864349
0.121386051178


In [59]:
def data_frame_to_lib_ffm(frame,key,field_list): #no le currenlty
    new_frame=pd.DataFrame()
    
    for column_name in frame.columns:
        if column_name!=key:
            col=[str(field_list[column_name])+':'+str(row)+':1' for row in frame[column_name]]         
        else:
            col=frame[column_name]
        new_frame[column_name]=col
    return new_frame

In [69]:
#convert to libffm
t=time.time()
lib_train_frame=data_frame_to_lib_ffm(final_frame,'clicked',{'clicked':-1, 'uuid':0, 'ad_id':1})
print time.time()-t
t=time.time()
lib_val_frame=data_frame_to_lib_ffm(final_val_frame,'clicked',{'clicked':-1, 'uuid':0, 'ad_id':1})
print time.time()-t


211.718625069
50.5209860802


In [70]:
t=time.time()
lib_tr ain_frame[['clicked','uuid','ad_id']].to_csv('train_frame.csv',sep=' ',header=None,index=False)
print time.time()-t
t=time.time()
lib_val_frame[['clicked','uuid','ad_id']].to_csv('val_frame.csv',sep=' ',header=None,index=False)
print time.time()-t

86.4987819195


KeyError: ('clicked', 'uuid', 'ad_id')

In [4]:
#greedy algorithm

# ----------- 
# model training goes here ... let's borrow from the "pandas is cool" script as a quick example
# ----------- 

#train

t=time.time()
ad_likelihood = train.groupby('ad_id').clicked.agg(['count','sum' ]).reset_index()
print time.time()-t

M   = train.clicked.mean()

ad_likelihood['likelihood'] = (ad_likelihood['sum'] + 12*M) / (12 + ad_likelihood['count'])


12.9538419247


In [ ]:
#pandas to libffm...

In [5]:
#predict
t=time.time()
val = val.merge(ad_likelihood, how='left')
print time.time()-t

val.likelihood.fillna(M, inplace=True)

t=time.time()
val.sort_values(['display_id', 'likelihood'], inplace=True, ascending=[True, False] )
print time.time()-t
val.head(3)

4.09866094589
9.24489688873


,display_id,ad_id,clicked,count,sum,likelihood
0,12,35982,0,104.0,34.0,0.313137
3,12,97178,1,7905.0,1623.0,0.205295
5,12,211592,0,70561.0,9833.0,0.139364


In [16]:
#calc mapk on validation set
util.fast_mapk(val) #columns: display id, likelihood,clicked

0.0892560482025
0.625658035278
1.22828912735
0.0125839710236
0.025496006012
MAP: 0.644582740275
